<a href="https://colab.research.google.com/github/bogatovam/cv-hse/blob/main/CV_HW_25_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Подготовка окружения

In [1]:
import cv2
import numpy as np

## Задание №1

В стереопаре известно положение эпиполя на втором изображении.
Кроме того, области на изображениях, соответствующие плоскому трехмерному объекту, связаны известным преобразованием гомографии. Найти выражение для фундаментальной матрицы через эпиполь и матрицу гомографии.

**Решение.**

По определению преобразование гомографии:
$ w\begin{pmatrix} u'\\v'\\1\end{pmatrix} =  H \begin{pmatrix} u\\v\\1\end{pmatrix}$


$ \begin{pmatrix} u\\v\\1\end{pmatrix} =  P \begin{pmatrix} X\\Y\\Z\\1\end{pmatrix};       \begin{pmatrix} u'\\v'\\1\end{pmatrix} =  P' \begin{pmatrix} X\\Y\\Z\\1\end{pmatrix}$

$ P^+\begin{pmatrix} u\\v\\1\end{pmatrix} =  \begin{pmatrix} X\\Y\\Z\\1\end{pmatrix} = P'^+\begin{pmatrix} u'\\v'\\1\end{pmatrix}$

$ \begin{pmatrix} u'\\v'\\1\end{pmatrix} =  P'P^+ \begin{pmatrix} u\\v\\1\end{pmatrix}$


Таким образом, фундаментальную матрицу можно записать в виде: 
> $F = [e']_XH $

> $H = P'P^+ $


## Задание №2

 Эпиполь на изображении в стереопаре имеет координаты $(1,1,1)^T$.
Найти преобразование гомографии, переводящее его в $(1,0,0)^T$.
 

**Решение.** 

Воспользуемся известной матрицой $G$ для преобразования эпиполей на одном изображении.

Однако, нам необходимо сделать координату игрек равной нулю, поэтому выполним афинное преобразование и подвинем точку на -1 по оси игрек.


In [2]:
G = [[1, 0,   0],
     [0, 1,  0],
     [-1, 0,  1]]

G = np.array(G)

T = [[ 1.,  0.,  0.],
     [ 0.,  1., -1.],
     [ 0.,  0.,  1.]]

T = np.array(T)

In [3]:
G_ = G.dot(T)
print("Homography matrix:\n{}\n".format(G_))
print("{}^T\t=>\t{}^T".format([[1,1,1]], G_.dot(np.array([[1,1,1]]).T).T))

Homography matrix:
[[ 1.  0.  0.]
 [ 0.  1. -1.]
 [-1.  0.  1.]]

[[1, 1, 1]]^T	=>	[[1. 0. 0.]]^T


## Задание №3

Вторая камера в стереопаре смещена относительно первой по оси x на величину T (в системе отсчета, связанной с первой камерой). Известны координаты пикселей, соответствующих трехмерной точке Q на обоих изображениях: (u,v) и
(u‘,v’) соответственно. Кроме того, известно, что камеры имеют разные внутренние параметры. Найти координату Z точки Q в системе отсчета первой камеры.

**Решение.** 
Известно, что:
> $(u,v,1) = P*(X,Y,Z,1)$

Запишем матрицы проекций для каждой камеры
> $P = K [I|0]$ 

> $P'= K'[I|T] $ 

Подставим матрицу проекции первой камерв в первое уравнение и распишем выражение для $x$-координаты $u$:
>$u = f_x \frac{X}{Z} + C_x (1)$

Аналогично для второй камеры:
>$u' = f_x'\frac{X + T_x}{Z} + C_x' (2)$

Вычтем первое уравнение из второго и преобразуем:
>$ (2) - (1): u' - u = f_x'\frac{X + T_x}{Z}- f_x\frac{X}{Z} +(C_x'-C_x)$

>$ (u' - u) - (C_x'-C_x) = \frac{ f_x'(X + T_x) - f_xX}{Z}$

>$ Z = \frac{f_x'(X + T_x) - f_xX}{ (u' - u) - (C_x'-C_x)} =\frac{(f_x' -f_x)X  + f_x'T_x}{ (u' - u) - (C_x'-C_x)} $




## Задание №4

Написать программу, которая находит фундаментальную матрицу
стереопары по известным 8 соответствиям. В качестве теста взять стереопару с смещением вдоль оси x (в системе отсчета первой камеры) и единичными матрицами внутренних параметров. Соответствия сгенерировать, спроектировав 8
трехмерных точек, выбранных случайно, на оба изображения. 

**Решение.** 

[Алгоритм взят из википедии](https://en.wikipedia.org/wiki/Eight-point_algorithm)

Выражение $({\mathbf  {y}}')^{{T}}\,{\mathbf  {F}}\,{\mathbf  {y}}=0$ можно преобразовать как ${\mathbf  {e}}\cdot {\tilde  {{\mathbf  {y}}}}=0$, где 

$  {\tilde  {{\mathbf  {y}}}} = \begin{pmatrix}y'_{1}y_{1}\\y'_{1}y_{2}\\y'_{1}\\y'_{2}y_{1}\\y'_{2}y_{2}\\y'_{2}\\y_{1}\\y_{2}\\1\end{pmatrix}$, ${\mathbf  {e}}={\begin{pmatrix}e_{{11}}\\e_{{12}}\\e_{{13}}\\e_{{21}}\\e_{{22}}\\e_{{23}}\\e_{{31}}\\e_{{32}}\\e_{{33}}\end{pmatrix}}$

и впоследствии решать как однородную систему уравнений, используя сингулярное разложение. 

Таким образом, каждую пару точек записываем в виде вектора $ {\tilde  {{\mathbf  {y}}}}$. Каждый получившийся вектор складывем в матрицу, по столбцам и решаем однородную систему уравнений.

In [4]:
points = []
num_points = 8

for i in range(num_points):
  point = np.random.rand(3) * 10
  points.append(point)

points = np.concatenate((points, np.ones((num_points, 1))), axis=1)
points

array([[0.58924404, 2.83834991, 5.04996497, 1.        ],
       [8.41593965, 1.82966809, 4.45578383, 1.        ],
       [4.17945907, 3.68499629, 8.80016175, 1.        ],
       [1.94308352, 7.03846162, 0.73675762, 1.        ],
       [9.5488857 , 0.51465053, 6.28366588, 1.        ],
       [8.02308615, 0.59941825, 0.54268065, 1.        ],
       [8.97753059, 2.90554951, 6.14699039, 1.        ],
       [2.77778317, 9.10919974, 5.01164164, 1.        ]])

In [5]:
T_1 = np.array([[0, 0, 0]]).T
T_2 = np.array([[10, 0, 0]]).T

R_1 = np.diag(np.ones((3, )))
R_2 = np.diag(np.ones((3, )))

P_1 = np.concatenate((R_1, T_1), axis=1)
P_2 = np.concatenate((R_2, T_2), axis=1)

In [6]:
points_1 = P_1.dot(points.T).T
points_2 = P_2.dot(points.T).T
points_1, points_2

(array([[0.58924404, 2.83834991, 5.04996497],
        [8.41593965, 1.82966809, 4.45578383],
        [4.17945907, 3.68499629, 8.80016175],
        [1.94308352, 7.03846162, 0.73675762],
        [9.5488857 , 0.51465053, 6.28366588],
        [8.02308615, 0.59941825, 0.54268065],
        [8.97753059, 2.90554951, 6.14699039],
        [2.77778317, 9.10919974, 5.01164164]]),
 array([[10.58924404,  2.83834991,  5.04996497],
        [18.41593965,  1.82966809,  4.45578383],
        [14.17945907,  3.68499629,  8.80016175],
        [11.94308352,  7.03846162,  0.73675762],
        [19.5488857 ,  0.51465053,  6.28366588],
        [18.02308615,  0.59941825,  0.54268065],
        [18.97753059,  2.90554951,  6.14699039],
        [12.77778317,  9.10919974,  5.01164164]]))

In [7]:
def normalize(x):
  return [(x[0][0]/x[0][2], x[0][1]/x[0][2]), (x[1][0]/x[1][2], x[1][1]/x[1][2])]

In [8]:
points_list = np.array(list(map(normalize, zip(points_1, points_2))))
points_list

array([[[ 0.1166828 ,  0.56205338],
        [ 2.09689455,  0.56205338]],

       [[ 1.88876749,  0.41062766],
        [ 4.13304154,  0.41062766]],

       [[ 0.4749298 ,  0.41874188],
        [ 1.61127255,  0.41874188]],

       [[ 2.63734433,  9.55329334],
        [16.21032924,  9.55329334]],

       [[ 1.51963613,  0.08190291],
        [ 3.11106384,  0.08190291]],

       [[14.78417574,  1.10455062],
        [33.21121924,  1.10455062]],

       [[ 1.46047578,  0.47267839],
        [ 3.08728815,  0.47267839]],

       [[ 0.55426612,  1.81760796],
        [ 2.54962028,  1.81760796]]])

In [9]:
def findFundamnentalMatrix(points_list):
  A = []
  for points in points_list:
    x, y = points[0]
    u, v = points[1]
    A.append([u*x, u*y, u, x*v, y*v, v, x, y, 1])

  A = np.asarray(A, dtype=np.float)
  A = A.T

  u, s, vt =  np.linalg.svd(A)
  print("Minimal Singular Value: {}\n".format(s[-1]))
  x_arr = u[:, -1]
  h_i = np.reshape(x_arr, (3,3))
  print("Fundamental matrix:\n{}\n".format(h_i))
  test = x_arr.dot(A)
  test[test < 1e-10] = 0
  print("The next values must be ZERO: {}\n\n".format(test))
  return h_i

In [10]:
H = findFundamnentalMatrix(points_list)

Minimal Singular Value: 0.012888433417570863

Fundamental matrix:
[[-5.69600561e-17  3.86622200e-15 -1.38123697e-16]
 [-1.04212697e-14 -4.35983988e-15 -7.07106781e-01]
 [ 4.42344949e-15  7.07106781e-01 -6.12587839e-15]]

The next values must be ZERO: [0. 0. 0. 0. 0. 0. 0. 0.]




In [11]:
q = np.concatenate((points_list[0][0], [1]))
q_ = np.concatenate((points_list[0][1], [1]))
q, q_

(array([0.1166828 , 0.56205338, 1.        ]),
 array([2.09689455, 0.56205338, 1.        ]))

In [12]:
q_.T.dot(H).dot(q)

5.551115123125783e-17